In [35]:
import torch
from torch import nn
import numpy as np

# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=2, padding=1, bias=False)
conv2d.weight = nn.Parameter(torch.Tensor([range(4)]).view((1, 1, 2, 2)))
X = torch.Tensor([range(9)]).view((3, 3))
Y = comp_conv2d(conv2d, X)
print(f"Input: {X}, \nshape: {tuple(X.shape)}\n---")
print(f"Kernal: {torch.Tensor(conv2d.weight)}, \nshape: {tuple(conv2d.weight[0][0].shape)}\n---")
print(f"Output: {Y}, \nshape: {tuple(Y.shape)}\n---")

Input: tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]]), 
shape: (3, 3)
---
Kernal: tensor([[[[0., 1.],
          [2., 3.]]]], grad_fn=<AliasBackward0>), 
shape: (2, 2)
---
Output: tensor([[ 0.,  3.,  8.,  4.],
        [ 9., 19., 25., 10.],
        [21., 37., 43., 16.],
        [ 6.,  7.,  8.,  0.]], grad_fn=<ReshapeAliasBackward0>), 
shape: (4, 4)
---


In [50]:
import torch

def corr2d(X, K):
    """Compute 2D cross-correlation.

    Defined in :numref:`sec_conv_layer`"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(corr2d(x, k) for x, k in zip(X, K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

Y = corr2d_multi_in(X, K)
print(f"Input: {X}, \nshape: {tuple(X.shape)}\n---")
print(f"Kernal: {K}, \nshape: {tuple(K.shape)}\n---")
print(f"Output: {Y}, \nshape: {tuple(Y.shape)}\n---")

Input: tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]]), 
shape: (2, 3, 3)
---
Kernal: tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]]), 
shape: (2, 2, 2)
---
Output: tensor([[ 56.,  72.],
        [104., 120.]]), 
shape: (2, 2)
---


In [51]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2), 0)
Y = corr2d_multi_in_out(X, K)

print(f"Input: {X}, \nshape: {tuple(X.shape)}\n---")
print(f"Kernal: {K}, \nshape: {tuple(K.shape)}\n---")
print(f"Output: {Y}, \nshape: {tuple(Y.shape)}\n---")

Input: tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]]), 
shape: (2, 3, 3)
---
Kernal: tensor([[[[0., 1.],
          [2., 3.]],

         [[1., 2.],
          [3., 4.]]],


        [[[1., 2.],
          [3., 4.]],

         [[2., 3.],
          [4., 5.]]],


        [[[2., 3.],
          [4., 5.]],

         [[3., 4.],
          [5., 6.]]]]), 
shape: (3, 2, 2, 2)
---
Output: tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]]), 
shape: (3, 2, 2)
---
